In [ ]:
#update if neccessary
import os
iskaggle = os.environ.get('KAGGLE_KERNAL_RUN_TYPE', '')

if iskaggle:
  !pip install -Uqq fastai

In [ ]:
#define the image fetching func

from fastcore.all import *
import time
import json

def search_images(term, max_images=100):
  url = 'https://duckduckgo.com/'
  res = urlread(url,data={'q':term})
  time.sleep(2)
  searchObj = re.search(r'vqd=([\d-]+)\&', res)
  requestUrl = url + 'i.js'
  params = dict(l='us-en', o='json', q=term, vqd=searchObj.group(1), f=',,,', p='1', v7exp='a')    
  headers = dict( referer='https://duckduckgo.com/' )
  urls,data = set(),{'next':1}
  while len(urls)<max_images and 'next' in data:
      res = urlread(requestUrl, data=params, headers=headers)
      data = json.loads(res) if res else {}
      urls.update(L(data['results']).itemgot('image'))
      requestUrl = url + data['next']
      time.sleep(4)
  return L(urls)[:max_images]

In [ ]:
#fetch images of mushrooms
from fastdownload import download_url
mushroom_types = 'blue oyster mushroom', 'shiitake mushroom', 'lion\'s mane mushroom', 'amanita muscaria mushroom'
path = Path('mushrooms')

for o in mushroom_types:
  dest = (path/o)
  dest.mkdir(exist_ok=True, parents=True)
  download_images(dest, urls=search_images(f'{o} photo'))
  time.sleep(5)
  resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
#unlink the bad images
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
#define the datablock
mushrooms = DataBlock(
  blocks=(ImageBlock, CategoryBlock),
  get_items=get_image_files,
  splitter=RandomSplitter(valid_pct=0.2, seed=42),
  get_y=parent_label,
  item_tfms=Resize(128)
)

In [ ]:
#augment some images, display a few
mushrooms = mushrooms.new(
  item_tfms=Resize(128, min_scale=0.3),
  batch_tfms=aug_transforms()
)
dls = mushrooms.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
#train the model
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
#display the confusion matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
#hide_output
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#unlink any bad data we found
for i in cleaner.delete(): cleaner.fns[i].unlink()

In [ ]:
#move any mis-categorizations
for i,cat in cleaner.change(): shutil.move(str(cleaner.fns[i]), path/cat)

In [ ]:
#fetch some new test data to maunally verify against the model
test_urls = search_images(mushroom_types[0], max_images=1)
oyst_dest = 'blue-oyster-mushroom-example.jpg'
download_url(urls[0], oyst_dest, show_progress=False)

test_im = Image.open(oyst_dest)
test_im.to_thumb(256,256)

In [ ]:
def print_prediction(img):
  pred,_,probs = learn_inf.predict(img)
  print(f"This is a {pred}, probability: {probs[0]:.4f}")

print_prediction(test_im)